
**Segment and Cluster Neighbourhoods in Toronto**
**IBM/Coursera Applied Datascience Capstone**
This Project Notebook assignment is Part 1 of 3 for Week 3: Applied Datascience Capstone Course assignment.

Load url into data objects for analysis:

Step 1: Build code to scrape following wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the Postal Code, Borough, and Neighborhood data.**bold text**

In [1]:
import pandas as pd
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")


Wrangle and clean data:

Step 2: Evaluate each of the data table objects in turn that we created during url loading.

Step 3: Designate primary dataframe and populate with required data objects loaded from url.

In [4]:
# Explore first few rows of our initial data[0] table
# Create df, primary dataframe, merged from data[0].
# Designate df as primary dataframe. 

df = data[0]
df.head(5)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"



Summary:
The first dataframe object data[0] created by html reader has the 3 column names we require, and is populated with 180 rows of data.
data[1] and data[2], our second and third imported dataframe objects, were both created during scraping by read_html and evaluated to see if any data is required of them. Both of the data sets contain Canada province/state higher level information, not in scope for our Toronto City level model, therefore are not used.
Steps 1 thru 3 are complete.
Step 4: Ensure Borough (town) cells with an assigned borough are processed. Cells where borough = 'Not assigned' are ignored.

In [6]:
print(df['Borough'].value_counts())

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64


In [9]:
df = df[df['Borough']!= 'Not assigned']
print(df['Borough'].value_counts())


North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64



Step 5: When two or more Neighborhoods share the same postal code, combine each Neighbourhood to a single neighborhood line comma delimited.

In [10]:

# I first perform a count sum of any df column duplicates by creating a lambda function. 
dup_chk = df.apply(lambda col: col.duplicated()).sum()
print(dup_chk)
print(df.head(5))

Postal Code       0
Borough          93
Neighbourhood     4
dtype: int64
  Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government



Step 6: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [11]:
#List all rows where Neighbourhood = 'Not assigned'. 
Num_Neighbourhood_Not_Assigneds = df[df['Neighbourhood'] == 'Not assigned']
Num_Neighbourhood_Not_Assigneds.head(5)

,Postal Code,Borough,Neighbourhood


In [12]:
#Create a Wk3:PostalDataPart1.csv file we will use for Wk3: Part 2.
df.to_csv('Wk3PostalDataPart1.csv', index=False)

#Test csv load
df2 = pd.read_csv('./Wk3PostalDataPart1.csv')
df2.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"



Summary:
Net of 103 net rows of data, after 77 borough 'Not Assigned' rows were removed from initial 180 rows reported during initial step.
Dataframe is ready for integration with next Part 2 assignment.

In [13]:

print(df.shape, 'rows x cols')

(103, 3) rows x cols


In [14]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' toronto_geocsv
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [18]:
df = pd.merge(geocsv_data, df2, on='Postal Code')
df

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,York,Weston
99,M9P,43.696319,-79.532242,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [20]:
df = df[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
df.head(11)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [21]:
df.to_csv('Toronto.TASK_II_df.csv',index=False)

In [22]:
df.shape

(103, 5)